In [16]:
import os, time, io, json, sys
import urllib3
import boto3
import h3ronpy
import elevation
import rasterio

import geopandas as gpd
import pandas as pd
import numpy as np
import folium as flm
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.dataMisc as dMisc

from h3ronpy.pandas.vector import geodataframe_to_cells, cells_dataframe_to_geodataframe
from h3ronpy import ContainmentMode
from dotenv import load_dotenv
from shapely.geometry import shape, box
from geojson_pydantic import Feature, Polygon
from urllib3.exceptions import InsecureRequestWarning
from botocore import UNSIGNED
from botocore.config import Config
from pystac_client import Client
from tqdm.notebook import tqdm

sys.path.append("../../src")

import global_zonal
import h3_helper

urllib3.disable_warnings(InsecureRequestWarning)

def tPrint(s):
    """prints the time along with the message"""
    print("%s\t%s" % (time.strftime("%H:%M:%S"), s))

s3_client = boto3.client('s3', verify=False, config=Config(signature_version=UNSIGNED))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Define input variables
base_folder = "C:/WBG/Work/S2S/data/elevation"
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

h3_0_list = h3_helper.generate_lvl0_lists(
    6,
    return_gdf=True,
    buffer0=False,
    read_pickle=True,
    pickle_file="h0_dictionary_of_h6_geodata_frames_land.pickle"
)

Loading pickle file h0_dictionary_of_h6_geodata_frames_land.pickle: it exists True


In [24]:
# Download elevation data for the H3 cell
# Too many tiles, does not work
# elevation.clip(b, temp_elev_file)

# search throught the open SRTM S3 data from here
# https://registry.opendata.aws/terrain-tiles/
elev_bucket = 'elevation-tiles-prod'
footprints_file = os.path.join(base_folder, 'srtm_tiles.geojson')
if not os.path.exists(footprints_file):
    tPrint("Downloading SRTM tile footprints...")
    s3_client.download_file(
        elev_bucket,
        'v2/docs/footprints-2017-11-06.geojson.gz',
        footprints_file+".gz"
    )
    import gzip
    with gzip.open(footprints_file+".gz", 'rt') as f_in:
        geojson_data = json.load(f_in)
    with open(footprints_file, 'w') as f_out:
        json.dump(geojson_data, f_out)
    os.remove(footprints_file+".gz")
elev_footprints = gpd.read_file(footprints_file)
srtm_footprints = elev_footprints.loc[elev_footprints['source'] == 'srtm', :].copy()
srtm_footprints.head()

,id,filename,resolution,source,url,min_zoom,max_zoom,priority,approximate_zoom,geometry
74,None,N57W104.tif,90.0,srtm,http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N57W...,7,20,0,13,"MULTIPOLYGON (((-104.00014 58.00014, -104.0001..."
75,None,N21E051.tif,90.0,srtm,http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N21E...,7,20,0,13,"MULTIPOLYGON (((50.99986 22.00014, 50.99986 20..."
78,None,N16W013.tif,90.0,srtm,http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N16W...,7,20,0,13,"MULTIPOLYGON (((-13.00014 17.00014, -13.00014 ..."
79,None,S25E117.tif,90.0,srtm,http://d1fl4qlsp9xnuu.cloudfront.net/srtm/S25E...,7,20,0,13,"MULTIPOLYGON (((116.99986 -23.99986, 116.99986..."
80,None,S44W177.tif,90.0,srtm,http://d1fl4qlsp9xnuu.cloudfront.net/srtm/S44W...,7,20,0,13,"MULTIPOLYGON (((-176.83343 -43.52778, -176.833..."


In [42]:
# Run zonal stats on a single H3 cell lvl 0
h0_id = list(h3_0_list.keys())[4]  # Use the first H3 cell ID from the list
h0_cells = h3_0_list[h0_id]
if h0_cells.crs != srtm_footprints.crs:    
    h0_cells = h0_cells.to_crs(srtm_footprints.crs)

# Identify rows in srtm_footprints that intersect with the parent H3 cell
joined_srtm = gpd.sjoin(srtm_footprints, h0_cells, how='inner', predicate='intersects') 
sel_srtm_path = joined_srtm['url'].unique().tolist()

all_res = []
for srtm_url in sel_srtm_path:
    tPrint(f"Processing SRTM tile: {srtm_url}")
    curR = rasterio.open(srtm_url)
    if h0_cells.crs != curR.crs:
        h0_cells = h0_cells.to_crs(curR.crs)
    elev_res = rMisc.zonalStats(h0_cells, curR, minVal=0, maxVal=10000, return_df=True)
    elev_res = elev_res.replace(-1, None)
    elev_res['shape_id'] = h0_cells['shape_id'].values
    all_res.append(elev_res)
    

10:53:23	Processing SRTM tile: http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N57E008.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\fea

10:53:41	Processing SRTM tile: http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N57E019.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\fea

10:54:01	Processing SRTM tile: http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N59E008.tif


...

c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\fea

11:11:37	Processing SRTM tile: http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N59E006.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\fea

11:11:59	Processing SRTM tile: http://d1fl4qlsp9xnuu.cloudfront.net/srtm/N56E008.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\fea

In [43]:
# Concatenate all results for the current h0
if len(all_res) > 0:
    cur_h0_res = pd.concat(all_res, ignore_index=True)
    cur_h0_res.set_index('shape_id', inplace=True)
    cur_h0_res['total_cells'] = cur_h0_res.sum(axis=1)
    cur_h0_res = cur_h0_res.loc[cur_h0_res['total_cells'] > 0]
    cur_h0_res['hex_id'] = cur_h0_res.index.values

    # If there are duplicates, sum the values for each hex_id
    cur_all_res = []
    for hex_id, curD in cur_h0_res.groupby('hex_id'):
        if len(curD) > 1:
            curD['MIN'] = curD['MIN'].min()
            curD['MAX'] = curD['MAX'].max()
            curD['MEAN'] = curD['MEAN'].mean()
            curD['hex_id'] = hex_id   
            curD = curD.to_frame().T             
        cur_all_res.append(curD)

    if len(cur_all_res) > 0:
        final_h0_res = pd.concat(cur_all_res)
        final_h0_res.to_parquet(out_lc_file, index=False)

,SUM,MIN,MAX,MEAN,shape_id
0,None,None,None,None,860800087ffffff
1,None,None,None,None,86080008fffffff
2,None,None,None,None,860800097ffffff
3,None,None,None,None,86080009fffffff
4,None,None,None,None,8608000b7ffffff
...,...,...,...,...,...
39461,None,None,None,None,8609abccfffffff
39462,None,None,None,None,8609abcd7ffffff
39463,None,None,None,None,8609abcdfffffff
39464,None,None,None,None,8609abcefffffff
